<a href="https://colab.research.google.com/github/guscassiano/Dio_ML_bootcamp/blob/voice_assistent/voice_assistent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**🤖 Projeto: Assistente de Voz com Python**
**Objetivo:** Criar um assistente de voz interativo capaz de entender comandos em português, executar tarefas automatizadas e responder usando uma voz sintetizada.

**Como funciona?**\
Este notebook está dividido em 5 passos principais:

1. **Configuração:** Instalamos todas as ferramentas (bibliotecas) que o nosso assistente precisa para funcionar.

2. **Módulos Principais:** Construímos as duas habilidades essenciais do assistente: a capacidade de falar (*Text-to-Speech*) e de ouvir (*Speech-to-Text*).

3. **Ações do Assistente:** Definimos as tarefas que ele pode executar, como pesquisar na Wikipedia, abrir o YouTube, etc.

4. **O Cérebro do Assistente:** Criamos a lógica principal que conecta o que ele ouve com a ação que ele deve tomar.

5. **Execução:** Iniciamos o assistente e começamos a interagir!

In [33]:
# ==============================================================================
# PASSO 1: CONFIGURAÇÃO DO AMBIENTE E INSTALAÇÃO DAS BIBLIOTECAS
# ==============================================================================
# Antes de começar, precisamos instalar todas as ferramentas necessárias.
# - gTTS: Para converter texto em áudio (a "voz" do assistente).
# - SpeechRecognition: Para converter nossa fala em texto (o "ouvido" do assistente).
# - wikipedia: Para permitir que o assistente faça pesquisas na Wikipedia.
# - mutagen: Para que o programa saiba a duração do áudio de resposta e espere ele terminar.
# - ffmpeg: Uma ferramenta poderosa para converter o formato do áudio gravado.

print("Instalando bibliotecas necessárias...")
!pip install gTTS SpeechRecognition wikipedia mutagen &> /dev/null
!apt-get -qq install -y ffmpeg &> /dev/null
print("Instalação concluída com sucesso!")

Instalando bibliotecas necessárias...
Instalação concluída com sucesso!


## **Passo 2: Construindo os Módulos Principais**
Agora vamos criar as duas funções mais importantes: `falar()` e `ouvir()`.\n

### **🔊 Módulo 1: A Voz do Assistente (Text-to-Speech)**
A função `falar` recebe um texto, o transforma em um áudio MP3 usando a API do Google, toca o áudio e, de forma inteligente, espera a fala terminar antes de continuar o programa.

In [34]:
# ==============================================================================
# MÓDULO 1: TEXT-TO-SPEECH (FALAR)
# ==============================================================================
from gtts import gTTS
from IPython.display import Audio, display
from mutagen.mp3 import MP3
import time
import subprocess

def falar(texto):
    """
    Converte uma string de texto em áudio, reproduz e espera o áudio terminar.
    """
    try:
        print(f"Assistente: {texto}")

        tts = gTTS(text=texto, lang='pt')
        tts.save('resposta.mp3')

        audio = MP3('resposta.mp3')
        duracao_audio = audio.info.length
        display(Audio('resposta.mp3', autoplay=True))

        # Isso evita que o assistente comece a ouvir enquanto ainda está falando
        time.sleep(duracao_audio + 0.5)

    except Exception as e:
        print(f"Ocorreu um erro no módulo de fala: {e}")

### **🎤 Módulo 2: O Ouvido do Assistente (Speech-to-Text)**
A função `ouvir` é um pouco mais complexa. Ela usa um código JavaScript para gravar o áudio do seu microfone diretamente no Colab, converte esse áudio para um formato compatível (WAV) e, finalmente, usa a API de reconhecimento de fala do Google para transcrever o que você disse para texto.

In [35]:
# ==============================================================================
# MÓDULO 2: SPEECH-TO-TEXT (OUVIR)
# ==============================================================================
import speech_recognition as sr
from google.colab import output
from IPython.display import Javascript
from base64 import b64decode

r = sr.Recognizer()

# Snippet de JavaScript para gravar áudio no navegador devido ao Colab
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def ouvir():
    """
    Grava a fala do usuário, converte para WAV e transcreve para texto.
    """
    try:
        print("Ouvindo por 5 segundos... (fale agora!)")
        display(Javascript(RECORD))
        s = output.eval_js('record(5000)')

        b = b64decode(s.split(',')[1])
        with open('audio_raw', 'wb') as f:
            f.write(b)

        subprocess.run(['ffmpeg', '-i', 'audio_raw', '-ac', '1', '-ar', '16000', 'audio.wav', '-y', '-hide_banner', '-loglevel', 'error'], check=True)

        with sr.AudioFile('audio.wav') as source:
            audio = r.record(source)
            texto = r.recognize_google(audio, language='pt-BR')
            print(f"Você disse: {texto}")
            return texto.lower()

    except sr.UnknownValueError:
        falar("Desculpe, não consegui entender o que você disse.")
        return ""
    except sr.RequestError:
        falar("Desculpe, estou com problemas de conexão com o serviço de reconhecimento.")
        return ""
    except Exception as e:
        print(f"Ocorreu um erro inesperado no módulo de audição: {e}")
        return ""

## **Passo 3: Criando as Ações do Assistente**
Aqui nós definimos as funções que serão acionadas por comandos de voz. Cada função representa uma "habilidade" do nosso assistente.


In [36]:
# ==============================================================================
# MÓDULO 3: FUNÇÕES DE COMANDO (AS "HABILIDADES")
# ==============================================================================
import wikipedia
import webbrowser
import requests

def pesquisar_wikipedia(termo):
    """Pesquisa um termo na Wikipedia e fala um resumo."""
    try:
        print(f"Pesquisando '{termo}' na Wikipedia...")
        wikipedia.set_lang('pt')
        resultado = wikipedia.summary(termo, sentences=2)
        falar(resultado)
    except Exception as e:
        falar(f"Desculpe, não encontrei resultados para {termo}. Tente outro termo.")

def abrir_youtube():
    """Abre o YouTube."""
    falar("Abrindo o YouTube.")
    # No Colab, não é possível abrir uma aba diretamente. Em vez disso, geramos um link clicável.
    print("Clique no link para abrir: https://www.youtube.com")

def encontrar_farmacia_proxima():
    """Usa a localização por IP para gerar um link do Google Maps com farmácias próximas."""
    try:
        # Usa uma API gratuita para obter a localização aproximada baseada no seu IP
        response = requests.get('http://ip-api.com/json/').json()
        lat = response['lat']
        lon = response['lon']
        falar("Buscando por farmácias na sua localização aproximada.")
        url = f"https://www.google.com/maps/search/farmácia/@{lat},{lon},15z"
        print(f"Clique no link para ver as farmácias próximas: {url}")
    except Exception as e:
        falar("Desculpe, não consegui obter sua localização.")

## **Passo 4: O Cérebro do Assistente - Juntando Tudo**
A função `assistente` é o coração do nosso projeto. Ela executa um loop infinito onde:

1. Chama a função `ouvir()` para capturar um comando.

2. Verifica se o comando corresponde a alguma das palavras-chave (`"pesquisar"`, `"youtube"`, etc.).

3. Chama a função de ação correspondente.

4. O loop só é quebrado se você disser "encerrar" ou "parar".

In [37]:
# ==============================================================================
# MÓDULO 4: LOOP PRINCIPAL (O "CÉREBRO")
# ==============================================================================
def assistente():
    """
    Função principal que ativa o assistente e gerencia os comandos.
    """
    falar("Olá! Sou seu assistente virtual. Como posso te ajudar hoje?")

    while True:
        comando = ouvir()

        if comando:
            if "pesquisar por" in comando:
                termo = comando.replace("pesquisar por", "").strip()
                pesquisar_wikipedia(termo)

            elif "abrir o youtube" in comando:
                abrir_youtube()

            elif "farmácia mais próxima" in comando:
                encontrar_farmacia_proxima()

            elif "encerrar" in comando or "parar" in comando:
                falar("Encerrando o assistente. Até a próxima!")
                break
            else:
                falar("Desculpe, não entendi o comando. Poderia repetir?")

## **Passo 5: Hora de Iniciar a Conversa!**
Tudo pronto! Agora é só executar a célula abaixo para dar vida ao seu assistente.

**Lembre-se:**

- Você precisará permitir o uso do microfone no seu navegador quando solicitado.

- Fale de forma clara durante os 5 segundos de gravação.

**Comandos que você pode usar:**

- `"Pesquisar por [termo que você quer]"` (ex: "Pesquisar por Brasil")

- `"Abrir o YouTube"`

- `"Farmácia mais próxima"`

- `"Encerrar"` ou `"Parar"`

In [38]:
# ==============================================================================
# EXECUÇÃO DO ASSISTENTE
# ==============================================================================
# Ao executar esta célula, o assistente será iniciado.

assistente()

Assistente: Olá! Sou seu assistente virtual. Como posso te ajudar hoje?


Ouvindo por 5 segundos... (fale agora!)


<IPython.core.display.Javascript object>

Você disse: Ir ao YouTube
Assistente: Desculpe, não entendi o comando. Poderia repetir?


Ouvindo por 5 segundos... (fale agora!)


...

<IPython.core.display.Javascript object>

Assistente: Desculpe, não consegui entender o que você disse.


Ouvindo por 5 segundos... (fale agora!)


<IPython.core.display.Javascript object>

Você disse: encerrar encerrar
Assistente: Encerrando o assistente. Até a próxima!
